In [4]:
import pandas as pd
from sqlalchemy import create_engine

# Define your connection string
user = "root"
password = "Travel2323!!"
host = "127.0.0.1"
port = "3306"
db = "sakila"

from sqlalchemy import create_engine
connection_string = (f'mysql+mysqlconnector://{user}:{password}@{host}:{port}/{db}')

# Create an engine
engine = create_engine(connection_string)

# Test the connection by reading a table
query = 'SELECT * FROM actor LIMIT 10;'

# Use pandas to execute the query and read the data into a DataFrame
df = pd.read_sql(query, con=engine)

# Display the DataFrame
print(df)


   actor_id first_name     last_name         last_update
0         1   PENELOPE       GUINESS 2006-02-15 04:34:33
1         2       NICK      WAHLBERG 2006-02-15 04:34:33
2         3         ED         CHASE 2006-02-15 04:34:33
3         4   JENNIFER         DAVIS 2006-02-15 04:34:33
4         5     JOHNNY  LOLLOBRIGIDA 2006-02-15 04:34:33
5         6      BETTE     NICHOLSON 2006-02-15 04:34:33
6         7      GRACE        MOSTEL 2006-02-15 04:34:33
7         8    MATTHEW     JOHANSSON 2006-02-15 04:34:33
8         9        JOE         SWANK 2006-02-15 04:34:33
9        10  CHRISTIAN         GABLE 2006-02-15 04:34:33


In [5]:
def rentals_month(engine, month, year):

    """
    Retrieve rental data for a given month and year from the Sakila database.

    Parameters:
    engine: SQLAlchemy engine object for database connection.
    month: Integer representing the month (1-12).
    year: Integer representing the year (e.g., 2024).

    Returns:
    A Pandas DataFrame containing the rental data for the specified month and year.
    """
    # Define the SQL query
    query = f"""
    SELECT *
    FROM rental
    WHERE MONTH(rental_date) = {month}
      AND YEAR(rental_date) = {year}
    """

    # Execute the query and return the results as a DataFrame
    df = pd.read_sql(query, con=engine)
    return df

In [6]:
def rental_count_month(df, month, year):
    """
    Count the number of rentals made by each customer during the specified month and year.
    
    Parameters:
    df: DataFrame containing rental data.
    month: Integer representing the month (1-12).
    year: Integer representing the year (e.g., 2024).
    
    Returns:
    A DataFrame with customer_id and the count of rentals for the specified month and year.
    """
    # Format the column name based on the provided month and year
    column_name = f"rentals_{month:02d}_{year}"
    
    # Group by customer_id and count the number of rentals
    rental_counts = df.groupby('customer_id').size().reset_index(name=column_name)
    
    return rental_counts

In [7]:
def compare_rentals(df1, df2):
    """
    Compare the number of rentals made by each customer in two different months and years.
    
    Parameters:
    df1: DataFrame containing the number of rentals made by each customer in the first month/year.
    df2: DataFrame containing the number of rentals made by each customer in the second month/year.
    
    Returns:
    A combined DataFrame with a new 'difference' column representing the difference in rentals.
    """
    # Merge the two DataFrames on customer_id
    df_combined = pd.merge(df1, df2, on='customer_id', how='outer', suffixes=('_month1', '_month2')).fillna(0)
    
    # Extract column names for rental counts
    rental_col_month1 = df1.columns[1]
    rental_col_month2 = df2.columns[1]
    
    # Calculate the difference between the two rental counts
    df_combined['difference'] = df_combined[rental_col_month2] - df_combined[rental_col_month1]
    
    return df_combined

In [17]:
rental_count_month

<function __main__.rental_count_month(df, month, year)>